In [ ]:
import os

from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Sequence
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.engine.url import URL

from sqlalchemy.dialects.postgresql import \
    ARRAY, BIGINT, BIT, BOOLEAN, BYTEA, CHAR, CIDR, DATE, \
    DOUBLE_PRECISION, ENUM, FLOAT, HSTORE, INET, INTEGER, \
    INTERVAL, JSON, JSONB, MACADDR, MONEY, NUMERIC, OID, REAL, SMALLINT, TEXT, \
    TIME, TIMESTAMP, UUID, VARCHAR, INT4RANGE, INT8RANGE, NUMRANGE, \
    DATERANGE, TSRANGE, TSTZRANGE, TSVECTOR

#Database connection from envirionment variables
connect_url = URL('postgres',
                  host=os.environ['crash_db_host'],
                  port=os.environ['crash_db_port'],
                  username=os.environ['crash_db_user'],
                  password=os.environ['crash_db_password'],
                  database=os.environ['crash_db'])

db = create_engine(connect_url)  
base = declarative_base()

In [ ]:
# Classs to manage meta experiments
class Meta(base):  
    __tablename__ = 'crash_meta'
    id = Column(BIGINT, Sequence('crash_meta_id_seq'), primary_key=True)
    experiments = Column(ARRAY(BIGINT), nullable=False)

In [ ]:
# Class to manage single experiment data
class Experiment(base):
    __tablename__ = 'crash_experiment'
    
    iterations = relationship("Iteration")
    results = relationship("Output")
    id = Column(BIGINT, Sequence('crash_experiment_id_seq'), primary_key=True)
    
    network = Column(JSONB, nullable=False)    
    no_of_edges = Column(INTEGER)
    no_of_nodes = Column(INTEGER)
    
    most_likely = Column(ARRAY(DOUBLE_PRECISION), nullable=False)
    optimistic = Column(ARRAY(DOUBLE_PRECISION), nullable=False)
    pessimistic = Column(ARRAY(DOUBLE_PRECISION), nullable=False)
    cov_mat = Column(ARRAY(DOUBLE_PRECISION), nullable=False)
    
    crash_cost = Column(ARRAY(DOUBLE_PRECISION), nullable=False)
    crash_time = Column(ARRAY(DOUBLE_PRECISION), nullable=False)
    
    penalty_b1 = Column(DOUBLE_PRECISION, nullable=False)
    penalty_m = Column(DOUBLE_PRECISION, nullable=False)
    penalty_steps = Column(INTEGER, nullable=False)
    t_final = Column(DOUBLE_PRECISION, nullable=False)
    t_init = Column(DOUBLE_PRECISION, nullable=False)
    penalty_type = Column(TEXT, nullable=False)
    
    kg_l = Column(DOUBLE_PRECISION)
    kg_sigma = Column(DOUBLE_PRECISION)
    kg_lambda = Column(JSONB)
    kg_mu = Column(JSONB)
    
    bootstrap = Column(BOOLEAN, nullable=False)
    confidence = Column(DOUBLE_PRECISION)
    resamples = Column(INTEGER)
    
    pareto_beta = Column(DOUBLE_PRECISION)
    
    scenarios_per_estimation = Column(INTEGER, nullable=False)
    total_scenarios = Column(INTEGER, nullable=False)
    method_type = Column(TEXT, nullable=False)
    
    seed = Column(INTEGER)
    seed_np = Column(INTEGER)
    
    network_figure = Column(BYTEA)
    network_pos = Column(JSONB)
    

In [ ]:
# Class to manage iteration data
class Iteration(base):
    __tablename__ = 'crash_iteration'
    
    id = Column(BIGINT, Sequence('crash_iteration_id_seq'), primary_key=True)
    exp_id = Column(BIGINT, ForeignKey('crash_experiment.id'), nullable=False)
    
    cov = Column(JSONB)
    kg_mu = Column(ARRAY(DOUBLE_PRECISION))
    kg_lambda = Column(ARRAY(DOUBLE_PRECISION))
    constr_tree = Column(JSONB)
    kg_e_tree = Column(ARRAY(DOUBLE_PRECISION))
    e_tree = Column(ARRAY(DOUBLE_PRECISION))
    std_tree = Column(ARRAY(DOUBLE_PRECISION))
    recordset_tree = Column(ARRAY(BOOLEAN))
    singleton_tree = Column(ARRAY(BOOLEAN)) 

In [ ]:
# Class to manage output data
class Output(base):
    __tablename__ = 'crash_output'
    
    id = Column(BIGINT, Sequence('crash_output_id_seq'), primary_key=True)
    exp_id = Column(BIGINT, ForeignKey('crash_experiment.id'), nullable=False)
    

In [ ]:
# Create session    
Session = sessionmaker(db)  
session = Session()

# Create 
base.metadata.create_all(db)

In [ ]:
# Insert an experiment
import json

experiment = Experiment(network=json.dumps({'a':1, 'b':2}), no_of_edges=2, no_of_nodes=19)

session.add(experiment)  
session.commit()

In [ ]:
# Insert an iteration
import json

iteration = Iteration(exp_id=experiment.id,
                      cov = json.dumps({'a':1, 'b':2}),
                      kg_mu = [1.2, 3.4],
                      kg_lambda = [2.4,62.0],
                      constr_tree = json.dumps({'a':1, 'd':2}),
                      kg_e_tree = [2.4,62.0],
                      e_tree = [2.4,62.0],
                      std_tree = [2.4,62.0],
                      recordset_tree = [True,False],
                      singleton_tree = [True])

session.add(iteration)  
session.commit()

In [3]:
import json
json.dumps([{'a':1, 'b':2}])

'[{"a": 1, "b": 2}]'

In [ ]:
# Insert 
experiment_run = Meta(experiments=[12,1223,61252,123])
session.add(experiment_run)  
session.commit()


# Get returned primay key value of just inserted row
print(experiment_run.id)


# Read
# I need to figure out the querries later!
runs = session.query(Meta)  
for run in runs:  
    print(run.experiments)

In [ ]:
# Close session
session.close()